In [87]:
# http://ediy.com.my/index.php/blog/item/34-connecting-arduino-to-uln2803-line-driver
# Using a ULN2803A transistor bus https://www.sparkfun.com/products/312
# D1 D2 D3 D4  pins 5 4 0 2


In [6]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [7]:
%sendtofile config.txt

wifiname        BV6000
wifipassword    bubblino
mqttbroker      192.168.43.1
boardname       wheredial1
delayus         1000
cycramp         3
rampdelayfac    0.5
pinled          2
wifialt         DoESLiverpool,decafbad00,10.0.100.1

Sent 9 lines (232 bytes) to config.txt.


In [22]:
%sendtofile main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine, math, sys, random

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
 
mpins = [ machine.Pin(p, machine.Pin.OUT)  for p in [5,4,0,2] ]
    
pinled.value(0)
time.sleep(2)
pinled.value(1)

delayus = int(fconfig["delayus"])
cycramp = int(fconfig["cycramp"])
rampdelayfac = float(fconfig["rampdelayfac"])
toothcurrent = 0  # 512 cycles per 10 teeth

seqplus = bytes(range(8))  # this is a cycle of 8
seqminus = bytes(range(7,-1,-1))

def advancetooth(dtooth):
    global toothcurrent
    toothnext = toothcurrent + dtooth
    cyccurrent = int(math.floor(toothcurrent/10*512))
    cycnext = int(math.floor(toothnext/10*512))
    print(cycnext - cyccurrent)
    seq = (seqplus if cycnext >= cyccurrent else seqminus)
    dcyc = abs(cyccurrent - cycnext)
    for j in range(dcyc):
        ldelayus = int(delayus*(1 + rampdelayfac*(max(0, cycramp-j, j+cycramp+1-dcyc))))
        for i in seq:
            mpins[(i//2 + (1 if (i%2) == 1 else 3))%4].value(1-(i%2))    
            time.sleep_us(ldelayus)
    toothcurrent = toothnext
    
def clearvolts():
    for d in mpins:
        d.value(0)

def wiggle(n):
    for i in range(n):
        advancetooth(1 if ((i%2)==0) else -1)
    clearvolts()

wiggle(2)
    
async def advanceteethtask(dteeth):
    for k in range(abs(dteeth)):
        advancetooth(1 if dteeth>0 else -1)
        await asyncio.sleep_ms(1)
    clearvolts()
    await client.publish(boardname+"/toothcurrent", str(toothcurrent))
    
def advanceteeth(dteeth):
    aloop.create_task(advanceteethtask(dteeth))        
    
boardname = fconfig["boardname"]
async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))

async def serialrepltask():
    sreader = asyncio.StreamReader(sys.stdin)
    while True:
        cmd = await sreader.readline()
        await callbackcmdtask("serial", cmd)

async def onconnecttask(client):
    print("subscribing")
    await client.publish(boardname+"/ip", client._sta_if.ifconfig()[0], retain=True, qos=1)
    await client.publish(boardname+"/toothcurrent/onconnect", str(toothcurrent))
    await client.subscribe(boardname+'/cmd', 1)
    wiggle(4)

async def mqtttask(bflip=False):
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

def randomwiggletask():
    while True:
        await asyncio.sleep_ms(60000*2)
        wiggle(2*random.getrandbits(2))

config['subs_cb'] = callbackcmd
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['connect_coro'] = onconnecttask

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.create_task(mqtttask())
aloop.create_task(randomwiggletask())
aloop.run_forever()


Sent 116 lines (3588 bytes) to main.py.
